In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as pl
import os
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, Activation, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, LeakyReLU, concatenate
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow import clip_by_value
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
import pickle

In [ ]:
train_images = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Images\\train_images.parquet').read().to_pandas()
test_images = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Images\\test_images.parquet').read().to_pandas()
val_images = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Images\\val_images.parquet').read().to_pandas()

In [ ]:
train_labels = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Labels\\train_labels.parquet').read().to_pandas()
test_labels = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Labels\\test_labels.parquet').read().to_pandas()
val_labels = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Labels\\val_labels.parquet').read().to_pandas()

In [ ]:
input_shape = train_images.shape[1]

In [ ]:
dnn = Sequential()
dnn.add(Dense(2500, input_shape=(input_shape,), activation=LeakyReLU(alpha=0.05)))
dnn.add(Dropout(0.5))
dnn.add(Dense(500, activation=LeakyReLU(alpha=0.05)))
dnn.add(Dropout(0.5))
dnn.add(Dense(7, activation='softmax'))

In [ ]:
optimizer = optimizers.Adam(learning_rate=0.000075)

In [ ]:
dnn.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
dnn_history = dnn.fit(np.array(train_images), np.array(train_labels), validation_data=(np.array(test_images), np.array(test_labels)),
                      batch_size = int(0.01*train_images.shape[0]), epochs=200)

In [ ]:
dnn.save('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Models\\Dense\\Model.h5')
with open('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Models\\Dense\\Training History.pkl','wb') as outfile:
    pickle.dump(dnn_history.history, outfile)

In [ ]:
dnn_predicted_categories = np.argmax(dnn.predict(val_images), axis=1)

In [ ]:
val_labels = np.argmax(np.array(val_labels), axis=1)

In [ ]:
dnn_cm = confusion_matrix(dnn_predicted_categories, val_labels, labels=[0,1,2,3,4,5,6])

In [ ]:
dnn_cm

In [ ]:
dnn_accuracy = (dnn_cm[0,0]+dnn_cm[1,1]+dnn_cm[2,2]+dnn_cm[3,3]+dnn_cm[4,4]+dnn_cm[5,5]+dnn_cm[6,6])/np.sum(dnn_cm)
print(f'Dense-Net Accuracy: {round(100*dnn_accuracy,1)}%')